In [ ]:
%pip install semantic-link-labs --q

In [ ]:
import sempy_labs as labs
from pyspark.sql import Row
import datetime

StatementMeta(, 6f2e38a5-24b0-4093-ad93-7084a885aca9, 11, Finished, Available, Finished)

In [ ]:
def log_to_table(level, workspaceid, message):
    now = datetime.datetime.utcnow().isoformat()
    log_df = spark.createDataFrame([Row(timestamp=now, level=level, workspaceId=workspaceid, message=message)])
    log_df.write.format("delta").mode("append").saveAsTable("enableADLSGenWorkspace")

StatementMeta(, 6f2e38a5-24b0-4093-ad93-7084a885aca9, 12, Finished, Available, Finished)

In [ ]:
p_storage_account_name="pbnapgtrprodscus01adls"
current_user = mssparkutils.env.getUserName()
workspaces_df = spark.sql(f"Select DISTINCT Workspace_Id from models_discovery_status where Status = 'Success' and Size > 10")
log_to_table("INFO","NA","Started ADLS Gen2 Integration process")
for row in workspaces_df.collect():
   ws_id = row['Workspace_Id']
   try:
      labs.assign_workspace_to_dataflow_storage(dataflow_storage_account=p_storage_account_name, workspace=ws_id)
      msg = f"Storage account '{p_storage_account_name}' assigned successfully to workspace ID {ws_id}."
      print(msg)
      log_to_table("INFO",ws_id,msg)
   except Exception as e:
      msg = f"🔴Error assigning storage: {e}"
      log_to_table("ERROR",ws_id,msg)             
log_to_table("INFO","NA","Completed ADLS Gen2 Integration process")

In [ ]:
# Test case, in case you want to try out
# Before running ensure that public endpoint for the storage is enabled, otherwise we will get unknown error. 
#p_storage_account_name="forpbibackups"
#ws_id="fc9f8098-8d0c-45ca-9964-fabfd9b7381c"
#labs.assign_workspace_to_dataflow_storage(dataflow_storage_account=p_storage_account_name, workspace=ws_id)


StatementMeta(, 6f2e38a5-24b0-4093-ad93-7084a885aca9, -1, Cancelled, , Cancelled)